In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-30"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9924,2024-05-30,Paraguai Lnb,20:30,Sol De America,Ciudad Nueva,2.56,1.47,139.5,1.85,1.85,0.0,0.00,0.00,CbCzVNwo,0.390625,0.680272,0.540541,0.540541,0.070897,862.616,712.402521,0.825863,0.6,1.341641,2.236068,1950.00,14.760,8.806543,0.596649,-108.0,558.670,410.289305,0.734404,0.6,1.341641,2.236068,795.20,8.240,6.031007,0.731918,-97.0,75,56,26.00,14.20,0.000,0.000,0.382504,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9925,2024-05-30,Paraguai Lnb,21:15,Dep. San Jose,Felix Perez Cardozo,1.09,6.63,154.5,1.88,1.79,0.0,0.00,0.00,v54MhRVi,0.917431,0.150830,0.531915,0.558659,0.068261,996.626,1240.198193,1.244397,2.4,1.341641,0.559017,2002.00,10.762,13.171105,1.223853,98.0,180.042,129.627614,0.719985,1.8,1.643168,0.912871,57.68,2.296,1.783264,0.776683,29.0,91,56,22.00,1.03,0.000,0.000,1.014863,0.034681,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9926,2024-05-30,Porto Rico Bsn,21:00,Gigantes de Carolina,Cangrejeros,2.12,1.67,182.5,1.82,1.88,-1.5,1.87,1.74,8lQ6UIXN,0.471698,0.598802,0.549451,0.531915,0.070501,197.700,43.907822,0.222093,1.8,1.643168,0.912871,227.90,2.044,0.473001,0.231410,14.0,202.066,75.227179,0.372290,1.2,1.643168,1.369306,185.92,2.160,0.622294,0.288099,7.0,106,83,2.15,2.24,168.388,231.168,0.167915,0.022933,0.050927,0.60,0.120,9.333333,0.000000,0.0,0.000000,0.49,0.098,6.836735,0.000000,0.0,0.000000
9927,2024-05-30,República Tcheca Nbl,13:00,Nymburk,Usti n. Labem,1.13,5.35,163.5,1.85,1.85,0.0,0.00,0.00,88naeNlA,0.884956,0.186916,0.540541,0.540541,0.071872,97.960,12.935422,0.132048,2.4,1.341641,0.559017,118.15,1.172,0.177257,0.151243,57.0,233.978,117.654409,0.502844,1.8,1.643168,0.912871,221.52,2.922,1.771333,0.606206,22.0,85,78,1.39,2.84,97.534,219.858,0.920985,0.000000,NaN,0.20,0.040,3.250000,0.953030,1.0,0.046970,-2.69,-0.538,-8.085502,0.465915,0.4,-0.065915
9928,2024-05-30,Rússia Liga Vtb United,13:30,Lokomotiv Kuban,Zenit Petersburg,1.69,2.19,165.5,1.80,1.99,-2.5,2.02,2.57,CzOkAQwo,0.591716,0.456621,0.555556,0.502513,0.048337,180.236,49.565001,0.275001,0.0,0.000000,NaN,235.20,2.188,0.382322,0.174736,-30.0,165.872,54.117856,0.326263,1.8,1.643168,0.912871,154.44,2.072,0.564066,0.272232,-17.0,96,99,2.45,1.56,122.308,161.612,0.182244,0.070897,0.169459,-0.87,-0.174,-3.965517,0.693168,0.6,-0.093168,0.01,0.002,595.000000,0.554316,0.5,-0.054316
9929,2024-05-30,Sérvia Superliga,13:00,Partizan,Mega Basket,1.23,3.84,170.5,1.85,1.85,0.0,0.00,0.00,lEKWVSBF,0.813008,0.260417,0.540541,0.540541,0.073425,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,88,92,1.29,3.37,0.000,0.000,0.728027,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9930,2024-05-30,Turquia Superliga,14:30,Fenerbahce,Besiktas,1.20,4.29,162.5,1.83,1.89,0.0,0.00,0.00,SOFE6Cr4,0.833333,0.233100,0.546448,0.529101,0.066434,127.708,37.973590,0.297347,3.0,0.000000,0.000000,117.00,1.216,0.263875,0.217002,98.0,161.664,76.148946,0.471032,2.4,1.341641,0.559017,131.97,2.010,1.401838,0.697432,68.0,100,83,1.17,1.59,107.784,193.092,0.795978,0.022810,NaN,-0.59,-0.118,-1.694915,0.891442,0.9,0.008558,-1.70,-0.340,-9.676471,0.558166,0.5,-0.058166
9931,2024-05

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
